# archive to parquet - partitioned data

Ailines data

In [ ]:
MLRUN_COMMIT = "0.4.5"

In [ ]:
!mlrun clean -p -r

In [1]:
import mlrun, os
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters

In [ ]:
ARTIFACT_PATH       = os.path.join(os.getcwd(), 'artifacts', '{{run.uid}}')

In [2]:
FUNCTION            = 'arc_to_parquet'
DESCRIPTION         = 'retrieve archive table and save as partitioned parquet dataset'

BASE_IMAGE          = f'mlrun/ml-base:ml:{MLRUN_COMMIT}'
JOB_KIND            = 'job'
TASK_NAME           = 'user-task-arc-to-part-parq'
FUNCTION_PY         = 'https://raw.githubusercontent.com/yjb-ds/functions/master/arc_to_parquet/arc_to_parquet.py'

ARCHIVE_BIG         = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears_10.csv"
ARCHIVE             = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears.csv"
ARCHIVE_SMALL       = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv"

USE_ARCHIVE         = ARCHIVE_SMALL

FILE_SHAPE          = (123_534_969, 21) # (rows, cols)
SMALL_FILE_SHAPE    = (43_978, 21)      # (rows, cols)

LOCAL_FILE_NAME     = 'airlines.pqt'
ARTIFACT_STORE_KEY  = 'airlines'

In [3]:
PARTS_DEST_FOLDER   = 'partitions'
PARTS_COLS          = ['Year', 'Month']

In [4]:
HEADER = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime',
          'UniqueCarrier','FlightNum','TailNum','ActualElapsedTime','CRSElapsedTime','AirTime',
          'ArrDelay','DepDelay','Origin','Dest','Distance','TaxiIn','TaxiOut','Cancelled',
          'CancellationCode','Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
          'LateAircraftDelay']

INC_COLS = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime',
            'UniqueCarrier','FlightNum', 'CRSElapsedTime','AirTime',
            'Origin','Dest','Distance', 'TaxiIn', 'TaxiOut']

ENCODING = 'latin-1'

DTYPES_COLS = {
     'CRSElapsedTime': 'float32', 
     'TailNum': 'str', 
     'Distance': 'float32',
     'TaxiIn' : 'float32',
     'TaxiOut': 'float32',
     'ArrTime': 'float32',
     'AirTime': 'float32',
     'DepTime':'float32', 
     'CarrierDelay': 'float32', 
     'WeatherDelay': 'float32', 
     'NASDelay':'float32', 
     'SecurityDelay':'float32', 
     'LateAircraftDelay':'float32'}

In [5]:
LABEL_COLUMN = "IsArrDelayed"

In [6]:
os.makedirs(os.path.join(ARTIFACT_PATH, PARTITIONS_DEST), exist_ok=True)

#### load function

In [7]:
func = mlrun.new_function(command=FUNCTION_PY, image=IMAGE, kind=JOB_KIND)
func.apply(mlrun.mount_v3io())

'ready'

In [8]:
# create and run the task
arc_to_parq_task = mlrun.NewTask(
    TASK_NAME, 
    handler=FUNCTION,  
    params={
        'archive_url': USE_ARCHIVE,
        'name'       : FILE_NAME, 
        'key'        : KEY,
        'dataset'    : PARTITIONS_DEST,
        'part_cols'  : PARTITION_COLS,
        'encoding'   : ENCODING,
        'inc_cols'   : INC_COLS,
        'dtype'      : DTYPES_COLS},
    artifact_path=ARTIFACT_PATH)
# run
run = func.run(arc_to_parq_task)

[mlrun] 2020-02-02 19:58:43,549 starting run user-task-arc-to-part-parq uid=b6c886203bee44b086753b496299cad2  -> http://mlrun-api:8080
[mlrun] 2020-02-02 19:58:43,628 Job is running in the background, pod: user-task-arc-to-part-parq-mvpxm
[mlrun] 2020-02-02 19:58:52,598 destination file does not exist, downloading
[mlrun] 2020-02-02 19:58:53,165 saved table to /User/mlrun/airlines/dataset-small/partitions
[mlrun] 2020-02-02 19:58:53,187 log artifact airlines at /User/mlrun/airlines/dataset-small/partitions, size: None, db: Y

[mlrun] 2020-02-02 19:58:53,201 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...99cad2,0,Feb 02 19:58:52,completed,function,host=user-task-arc-to-part-parq-mvpxmkind=jobowner=admin,,"archive_url=https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csvdataset=partitionsdtype={'AirTime': 'float32', 'ArrTime': 'float32', 'CRSElapsedTime': 'float32', 'CarrierDelay': 'float32', 'DepTime': 'float32', 'Distance': 'float32', 'LateAircraftDelay': 'float32', 'NASDelay': 'float32', 'SecurityDelay': 'float32', 'TailNum': 'str', 'TaxiIn': 'float32', 'TaxiOut': 'float32', 'WeatherDelay': 'float32'}encoding=latin-1inc_cols=['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'CRSElapsedTime', 'AirTime', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']key=airlinesname=airlines.pqtpart_cols=['Year', 'Month']target_path=/User/mlrun/airlines/dataset-small",,airlines


to track results use .show() or .logs() or in CLI: 
!mlrun get run b6c886203bee44b086753b496299cad2  , !mlrun logs b6c886203bee44b086753b496299cad2 
[mlrun] 2020-02-02 19:59:02,854 run executed, status=completed


## tests

### a partitioned parquet table

In [9]:
import os
import pandas as pd
import pyarrow.parquet as pq

In [10]:
dataset = pq.ParquetDataset(os.path.join(TARGET_PATH, PARTITIONS_DEST))
df = dataset.read().to_pandas()

In [11]:
df.set_index(PARTITION_COLS, inplace=True)

In [12]:
df.head()

DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  CRSArrTime  \
Year Month                                                                    
1987 10     14          3          741.0    730         912.0    849          
     10     15          4          729.0    730         903.0    849          
     10     17          6          741.0    730         918.0    849          
     10     18          7          729.0    730         847.0    849          
     10     19          1          749.0    730         922.0    849          

           UniqueCarrier  FlightNum  CRSElapsedTime  AirTime  ... Dest  \
Year Month                                                    ...        
1987 10     PS            1451       79.0           NaN       ...  SFO   
     10     PS            1451       79.0           NaN       ...  SFO   
     10     PS            1451       79.0           NaN       ...  SFO   
     10     PS            1451       79.0           NaN       ...  SFO   
     10     PS            1451       79.0           NaN       ...  SFO   

           Distance  TaxiIn  TaxiOut  Cancelled  CarrierDelay  WeatherDelay  \
Year Month                                                                    
1987 10     447.0   NaN     NaN       0         NaN           NaN             
     10     447.0   NaN     NaN       0         NaN           NaN             
     10     447.0   NaN     NaN       0         NaN           NaN             
     10     447.0   NaN     NaN       0         NaN           NaN             
     10     447.0   NaN     NaN       0         NaN           NaN             

            NASDelay  SecurityDelay  LateAircraftDelay  
Year Month                                              
1987 10    NaN       NaN            NaN                 
     10    NaN       NaN            NaN                 
     10    NaN       NaN            NaN                 
     10    NaN       NaN            NaN                 
     10    NaN       NaN            NaN                 

[5 rows x 21 columns]

In [13]:
if USE_ARCHIVE == ARCHIVE:
    assert df.shape==FILE_SHAPE
if USE_ARCHIVE == ARCHIVE_SMALL:
    assert df.shape==SMALL_FILE_SHAPE, f"{df.shape}"

In [14]:
df.shape

(43978, 21)

## cleanup

In [ ]:
# import shutil
# shutil.rmtree(TARGET_PATH)